In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, avg, to_timestamp, hour, date_format, month, year, when, mean, expr, element_at
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
key_filepath = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("US Accidents") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",key_filepath  ) \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/07 16:33:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [5]:

# Read Parquet files from GCS
parquet_files = [
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2016/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2017/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2018/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2019/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2020/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2021/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2022/933c14c388864f19a17c514e311a69b1-0.parquet",
                "gs://us-accidents-bucket/us_accidents_data/Start_Year=2023/933c14c388864f19a17c514e311a69b1-0.parquet"
                ]

df_list = [spark.read.option("header", "true").option("inferSchema", "true").parquet(file) for file in parquet_files]

merged_df = df_list[0]
for df in df_list[1:]:
    merged_df = merged_df.unionAll(df)



In [5]:
merged_df.show()

24/04/30 23:00:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------+--------+-------------------+-------------------+-----------------+------------------+------------+--------------------+--------------------+------------+----------+-----+-------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+--------+-------+-------+---------------+--------------+--------------+
|  ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|Distance(mi)|         Description|              Street|        City|    County|State|Country|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Weather_Condition|Crossing|Railway|Station|Traffic_Calming|Traffic_Signal|Sunrise_Sunset|
+----+-------+--------+-------------------+-------------------+-----------------+------------------+------------+--------------------+--------------------+------------+----------+-----+-------+--------------+-------------+-----------+

# Cleaning the data

### Steps:
> Removing unimportnant columns

> Renaming the columns

> Convert the values of the columns into easy handled values

In [6]:
from pyspark.sql.functions import to_timestamp

# Convert "Start_Time" and "End_Time" columns to datetime format
merged_df = merged_df.withColumn("Start_Time", to_timestamp(merged_df["Start_Time"], "yyyy-MM-dd HH:mm:ss"))
merged_df = merged_df.withColumn("End_Time", to_timestamp(merged_df["End_Time"], "yyyy-MM-dd HH:mm:ss"))


In [7]:

# Convert "Severity" column from string to integer
merged_df = merged_df.withColumn("Severity", merged_df["Severity"].cast(IntegerType()))


In [8]:
# TODO: If the rows that have at least one null value remove them, and if they are large number 
# replace the null with the average of its column
merged_df = merged_df.dropna()

In [39]:
merged_df.count()

5656839

## We have dropped about 2 millions records because of nulls :)

In [9]:
# Create a new column called Duration = End_Time - Start_Time
# To indicate the duration of the accident
from pyspark.sql.functions import col, unix_timestamp

# Convert "Start_Time" and "End_Time" columns to Unix timestamp (seconds since the epoch)
merged_df = merged_df.withColumn("Start_Time_unix", unix_timestamp("Start_Time"))
merged_df = merged_df.withColumn("End_Time_unix", unix_timestamp("End_Time"))

# Calculate the duration (in seconds)
merged_df = merged_df.withColumn("Duration", col("End_Time_unix") - col("Start_Time_unix"))


In [10]:
# Add columns for start_time
merged_df = merged_df.withColumn('Hour_Of_Acc', hour(merged_df['Start_Time']))
merged_df = merged_df.withColumn('Day_Of_Acc', date_format(merged_df['Start_Time'], 'EEEE'))
merged_df = merged_df.withColumn('Month_Of_Acc', month(merged_df['Start_Time']))

In [11]:
# Use 'when' function to assign temperature categories based on the bins
temperature_category_column = when(merged_df['Temperature(F)'] <= 50, 'Cold') \
    .when((merged_df['Temperature(F)'] > 50) & (merged_df['Temperature(F)'] <= 75), 'Moderate') \
    .otherwise('Warm')

# Add the temperature category column to the DataFrame
merged_df = merged_df.withColumn('Temperature_Category', temperature_category_column)

In [12]:
# Convert "Distance(mi)" column from miles to meters
merged_df = merged_df.withColumn("Distance(m)", col("Distance(mi)") * 1609.34)

In [13]:
# Drop unnecessary or redundant columns

# List of column names to drop
columns_to_drop = ['Country', 'Start_Time', 'End_Time', 'Distance(mi)', 'Start_Time_unix', 'End_Time_unix']

# Drop the unimportant columns
merged_df = merged_df.drop(*columns_to_drop)



In [37]:
merged_df.show()

+----+-------+--------+---------+------------------+--------------------+--------------------+------------+----------+-----+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+--------+-------+-------+---------------+--------------+--------------+--------+-----------+----------+------------+--------------------+-----------+
|  ID| Source|Severity|Start_Lat|         Start_Lng|         Description|              Street|        City|    County|State|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Weather_Condition|Crossing|Railway|Station|Traffic_Calming|Traffic_Signal|Sunrise_Sunset|Duration|Hour_Of_Acc|Day_Of_Acc|Month_Of_Acc|Temperature_Category|Distance(m)|
+----+-------+--------+---------+------------------+--------------------+--------------------+------------+----------+-----+--------------+-------------+-----------+------------+--------------+--------------+--------

In [42]:
merged_df.columns

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Weather_Condition',
 'Crossing',
 'Railway',
 'Station',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Distance(m)',
 'Start_Time_unix',
 'End_Time_unix',
 'Duration']

# Predictive Analytics

1. [Regression]➔ Predicting accident Duration as indicator of impact on
traffic flow.

2. [classification]➔ Predicting the severity of an accident based on the
factors involved.

### Based on the EDA phase we will use the following features in our model:
- Start_Lat
- Start_Lng
- Source
- Duration
- Hour_Of_Acc
- Day_Of_Acc
- Month_Of_Acc
- Temperature
- Distance
- State
- Crossing
- Railway
- Traffic_Calming
- Traffic_Signal

### Note: 

We execluded most of weather conditions as it does not affect on the severity as follows:  

Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).

In [14]:

columns_to_keep = [
    "Start_Lat", "Start_Lng",  'Source', "Duration", "Hour_Of_Acc", "Day_Of_Acc",
    "Month_Of_Acc", "Temperature(F)", "Distance(m)", "State", "City", "Crossing",
    "Railway", "Traffic_Calming", "Traffic_Signal", "Severity"
]

# Selecting only the desired columns
data_df = merged_df.select(columns_to_keep)

# Showing the resulting DataFrame
data_df.show(1)

24/05/07 16:34:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
|Start_Lat| Start_Lng| Source|Duration|Hour_Of_Acc|Day_Of_Acc|Month_Of_Acc|Temperature(F)|Distance(m)|State|        City|Crossing|Railway|Traffic_Calming|Traffic_Signal|Severity|
+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
|39.063148|-84.032608|Source2|    1800|          6|    Monday|           2|          36.0|    16.0934|   OH|Williamsburg|   false|  false|          false|          true|       2|
+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
only showing top 1 row



### Convert the string and boolean data into numerical for feature vector

In [15]:
from pyspark.sql import DataFrame

def convert_string_data(df: DataFrame, column_name: str) -> DataFrame:
    # Count the occurrences of each unique value
    value_counts = df.groupBy(column_name).count()

    # Sort by count in descending order and take the top 20
    top_20_values = value_counts.orderBy(col('count').desc()).limit(20)

    # Collect the top 20 values
    top_20_values_list = top_20_values.select(column_name).rdd.flatMap(lambda x: x).collect()

    # Create a new DataFrame to accumulate transformations
    new_df = df

    # Loop through each unique value and create a new column for each
    for source in top_20_values_list:
        new_column_name = source.replace(" ", "_").lower() + "_" + column_name
        # Accumulate transformations by using the new DataFrame
        new_df = new_df.withColumn(new_column_name, when(col(column_name) == source, 1).otherwise(0))

    # Drop the original column
    new_df = new_df.drop(column_name)

    return new_df

def convert_boolean_data(df: DataFrame, column_name: str) -> DataFrame:
    new_df = df
    new_df = new_df.withColumn(column_name, when(col(column_name), 1).otherwise(0))
    return new_df

In [16]:
# Convert boolean data
boolean_cols = [ 'Crossing', 'Railway',  'Traffic_Calming', 'Traffic_Signal']  
data_after_boolean = data_df
for column in boolean_cols:
    data_after_boolean = convert_boolean_data(data_after_boolean, column)
data_after_boolean.show(1)

+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
|Start_Lat| Start_Lng| Source|Duration|Hour_Of_Acc|Day_Of_Acc|Month_Of_Acc|Temperature(F)|Distance(m)|State|        City|Crossing|Railway|Traffic_Calming|Traffic_Signal|Severity|
+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
|39.063148|-84.032608|Source2|    1800|          6|    Monday|           2|          36.0|    16.0934|   OH|Williamsburg|       0|      0|              0|             1|       2|
+---------+----------+-------+--------+-----------+----------+------------+--------------+-----------+-----+------------+--------+-------+---------------+--------------+--------+
only showing top 1 row



In [17]:
# Convert String data
str_columns = ["Source", "Day_Of_Acc", "State", "City"]
data_after_str = data_after_boolean
for column in str_columns:
    data_after_str = convert_string_data(data_after_str, column)

data_after_str.show(1)

+---------+----------+--------+-----------+------------+--------------+-----------+--------+-------+---------------+--------------+--------+--------------+--------------+--------------+-----------------+-------------------+--------------------+------------------+-----------------+-------------------+-----------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------------+------------+--------------+-----------+------------+------------+--------------+-----------+----------------+---------------+----------------+------------+--------------+------------+-------------+---------------+-----------+-----------------+------------------+
|Start_Lat| Start_Lng|Duration|Hour_Of_Acc|Month_Of_Acc|Temperature(F)|Distance(m)|Crossing|Railway|Traffic_Calming|Traffic_Signal|Severity|source1_Source|source2_Source|source3_Source|friday_Day_Of_Acc|thursd

In [19]:
new_df = data_after_str

In [21]:
# Split the data
from pyspark.sql.functions import rand
# Define the split ratios
train_ratio = 0.8
test_ratio = 0.2

# Split the DataFrame into train and test
train_df, test_df = new_df.randomSplit([train_ratio, test_ratio], seed=42)

# Show the number of rows in each split
# print("Number of rows in df:",new_df.count())
# print("Number of rows in train set:", train_df.count())
# print("Number of rows in test set:", test_df.count())

In [22]:
# Get the index of 'price' column
severity_index = new_df.columns.index('Severity')
print("Index of 'Severity' column:", severity_index)# Get the index of 'Severity' column


Index of 'Severity' column: 11


In [23]:
# Define a function to extract features and label from each row
def extract_features_label(row, severity_index):
    features = row[:severity_index] + row[severity_index+1:]  # Extract all columns except the 'Severity' column
    label = row[severity_index]  # Extract the 'Severity' column
    return (features, label)
transformed_rdd = train_df.rdd.map(lambda row: extract_features_label(row, severity_index))

In [ ]:
transformed_rdd.first()

((33.183228,
  -117.296371,
  2673,
  21,
  12,
  44.1,
  16.0934,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 2)

## 1. Regression

In [24]:
from linear_regression_mr import linear_regression, predict


In [ ]:
learning_rate = 0.0001
num_iterations = 100
weights, cost_history = linear_regression(transformed_rdd, learning_rate, num_iterations)